# LangChain: Models, Prompts and Output Parsers(모델, 프롬프트, 그리고 출력 파서)


## 개요

 * OepnAI call을 직접 사용하기(이전 강의들과 동일)
 * LangChain을 이용하여 API를 사용하기:
   * 프롬프트
   * 모델
   * Output Parsers(출력 파서)

## 당신의 OpenAI Key를 불러오세요 [OpenAI API Key](https://platform.openai.com/account/api-keys)
로컬에서 실행할 때 라이브러리가 설치되어 있지 않다면 아래 라이브러리들을 설치해 줍니다.

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

OpenAI의 API를 불러와 직접 사용해보는 것부터 시작합시다.  
(사실 이전 강의들에선 이 방식으로만 구현했었기 때문에 이미 알고 있는 내용이죠 😉)

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [ ]:
get_completion("What is 1+1?")

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm and respectful tone
"""

<span style='color:red'>style 변수에 저장된 내용을 따라 입력 메세지를 변경해달라는 프롬프트</span>를 입력하고 있습니다.

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [ ]:
response = get_completion(prompt)

In [ ]:
response

## Chat API : LangChain

이제 LangChain을 이용하여 같은 내용을 구현해봅시다.  
(라이브러리가 없다면 설치/업그레이드)

In [ ]:
#!pip install --upgrade langchain

### 모델

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# LLM이 생성한 문장의 다양성을 조절하기 위해 temperature를 0.0으로 설정합니다.
# 항상 같은 답변만 생성하도록 하겠다는 뜻입니다. 반대로 1.0으로 설정하면 항상 다양한
# 결과물을 생성합니다.
chat = ChatOpenAI(temperature=0.0)
chat

### 프롬프트 탬플릿(양식)
반복적으로 사용될 프롬프트 탬플릿을 정해줍니다.  
시스템 메세지와 유저 메세지를 설정하던 것과 유사합니다.

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

LangChain에서 제공하는 함수를 사용하여 간편하게 템플릿을 구성할 수 있습니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
# LLM에게 고객의 메세지 스타일을 바꿔달라고 요청해보세요
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

정해진 탬플릿에 변수만 바꿔서 입력하면 됩니다.

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

## Output Parsers(출력 파서)

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)


In [ ]:
type(response.content)

In [ ]:
# gift는 딕셔너리에 포함되어있지 않기 때문에, 아래 코드를 실행하면 에러가 발생합니다.
# 'gift'는 string입니다.
response.content.get('gift')

### LLM의 출력을 파이썬 딕셔너리로 파싱해봅니다.

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

여기서도 LangChain이 제공하는 함수를 사용하여 간단히 구현할 수 있습니다.  
특히 여러 개의 schema를 한 리스트에 묶어 입력으로 제공하는 것에 주목하세요.

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()


In [ ]:
print(format_instructions)

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')